In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [5]:
# train/test/split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']),
                                                   df['Survived'],
                                                   test_size=0.2,
                                                   random_state=42)

In [6]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [7]:
y_train.sample(5)

756    0
173    0
679    1
189    0
302    0
Name: Survived, dtype: int64

In [8]:
# imputatinon Transformer
trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

- SimpleImputer is applied to column index 2 (for 'age') to fill missing values with the mean of non-missing values.

- For column index 6 (for 'embarked'), it uses SimpleImputer with the strategy 'most_frequent' to fill missing values with the most common value.

- The parameter remainder='passthrough' allows the columns not specified (other than index 2 and index 6) to pass through without any changes.

In [9]:
# one hot encoding
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

- Uses OneHotEncoder to transform categorical features into binary-encoded columns.
- The transformer is applied to column index 1 (for 'sex') and column index 6 (for 'embarked').
- sparse=False parameter ensures the output is a dense array instead of a sparse matrix.
- handle_unknown='ignore' allows handling unknown categories gracefully by ignoring them during encoding.
- The parameter remainder='passthrough' lets the columns not specified (other than index 1 and index 6) to pass through without any changes.

In [27]:
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,8))
])

- Uses MinMaxScaler to scale the numerical features within a specific range (by default, between 0 and 1).
- The scaler is applied to columns from index 0 to 7 (inclusive of index 0 but exclusive of index 8).
- It standardizes the numerical values within each column independently based on the minimum and maximum values observed in that column.

In [11]:
#Feature Scaling
trf4 = SelectKBest(score_func=chi2, k=5)

- SelectKBest is a method for selecting the k highest scoring features.
- score_func=chi2 specifies the chi-squared scoring function, which measures the dependence between variables.
- k=5 indicates that it selects the 5 best features based on their chi-squared scores. These will likely be the features most correlated with the target variable.

In [12]:
#train the model
trf5 = DecisionTreeClassifier()

# Create Pipleine

In [13]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5),
])

# Pipeline Vs make_pipeline

pipeline requires making of steps, make_pipeline does not.

(Same applies to ColumnTransformer vs make_column_transformer)

In [14]:
# Alternate Syntax
pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [25]:
# train
pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 8, None))])),
                ('selectkbest',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x000001B894F757E0>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

# Explore the Pipeline

In [29]:
#Display Pipeline
from sklearn import set_config
set_config(display='diagram')

In [17]:
#predict
y_pred = pipe.predict(X_test)

In [18]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6256983240223464

# Cross Validation using Pipeline

In [19]:
#cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

0.6391214419383433

# Exporting the Pipeline

In [24]:
#export

import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))